In [1]:
import sys
sys.path.append("../..")

import time
from time import sleep
from datetime import datetime

# Import necessary libraries
import MetaTrader5 as mt5
import pandas as pd
from IPython.display import display
from lightweight_charts import JupyterChart
from IPython.display import clear_output, display

from sesto.fractal import high_low_finder
from sesto.utils import get_price_at_pnl, calculate_fee, calculate_position_size, get_pnl_at_price
from sesto.metatrader.business import get_positions, send_market_order, modify_sl_tp
from sesto.metatrader.data import fetch_data_pos
from sesto.constants import MT5Timeframe

MetaTrader5 initialized successfully
MetaTrader 5 initialized successfully.


In [2]:
# PAIRS = ['NZDJPY', 'AUDJPY', 'CADJPY', 'USDJPY']
PAIRS = ['BITCOIN', 'ETHEREUM', 'SOLANA']
MAIN_TIMEFRAME = MT5Timeframe.M15

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25
CAPITAL_PER_TRADE = 200  # $200 per trade
LEVERAGE = 500
DEVIATION = 20

TRAILING_STOP_STEPS = [
    {'trigger_tp_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.15},
    {'trigger_tp_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_tp_multiplier': 0.10, 'new_sl_pnl_multiplier': 0.01},
]

In [6]:
point = mt5.symbol_info('BITCOIN').point
price = mt5.symbol_info_tick('BITCOIN').ask
order = send_market_order('BITCOIN', 0.1, 'sell', price + 1000 * point, price - 2000 * point, DEVIATION)
order

Order successfully placed: OrderSendResult(retcode=10009, deal=192815026, order=228083148, volume=0.1, price=63831.97, bid=0.0, ask=0.0, comment='Request executed', request_id=1092877708, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BITCOIN', volume=0.1, price=63831.97, stoplimit=0.0, sl=63841.97, tp=63821.97, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=192815026, order=228083148, volume=0.1, price=63831.97, bid=0.0, ask=0.0, comment='Request executed', request_id=1092877708, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BITCOIN', volume=0.1, price=63831.97, stoplimit=0.0, sl=63841.97, tp=63821.97, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

In [4]:
# size_usd = calculate_position_size(capital=CAPITAL_PER_TRADE, leverage=LEVERAGE)
# fee = calculate_fee(position_size_usd=size_usd) 

# while True:
#     clear_output(wait=True)

#     for pair in PAIRS:
#         df = fetch_data_pos(pair, MAIN_TIMEFRAME, 5)
#         df['fractal'] = high_low_finder(df)
#         df['symbol'] = pair
#         display(df.tail())

#         last_row = df.iloc[-1]

#         if df['fractal'].iloc[-1] == 'top':
#             sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='short')
#             tp = get_price_at_pnl(pnl_multiplier=TP_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='short')
#             order = send_market_order(pair, 1, 'sell', sl, tp, DEVIATION)
#             display(order)
#         elif df['fractal'].iloc[-1] == 'bottom':
#             sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='long')
#             tp = get_price_at_pnl(pnl_multiplier=TP_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='long')
#             order = send_market_order(pair, 1, 'buy', sl, tp, DEVIATION)
#             display(order)

#     positions = get_positions()

#     for index, position in positions.iterrows():
#         for trailing_step in TRAILING_STOP_STEPS:
#             trigger_tp_multiplier, new_sl_pnl_multiplier = trailing_step['trigger_tp_multiplier'], trailing_step['new_sl_pnl_multiplier']
#             pnl_threshold = TP_PNL_MULTIPLIER * trigger_tp_multiplier * CAPITAL_PER_TRADE

#             if position.profit >= pnl_threshold:
#                 old_sl_price = position['sl']

#                 if position.type == 0:
#                     new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=fee, position_size_usd=position.volume, leverage=LEVERAGE, entry_price=position.price_open, type='long')
#                     if new_sl_price > old_sl_price:
#                         modify_sl_tp(position.ticket, new_sl_price, position.tp)
#                 else:
#                     new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=fee, position_size_usd=position.volume, leverage=LEVERAGE, entry_price=position.price_open, type='short')
#                     if new_sl_price < old_sl_price:
#                         modify_sl_tp(position.ticket, new_sl_price, position.tp)
                
#                 if new_sl_price:
#                     capital_gain_at_current_unrealized_pnl = (position.profit / CAPITAL_PER_TRADE) * 100
#                     old_sl_price_diff = (position.price_open / old_sl_price - 1) * 100
#                     new_sl_price_diff = (position.price_open / position.sl - 1) * 100
#                     pnl_at_old_sl = get_pnl_at_price(current_price=old_sl_price, entry_price=position.price_open, position_size_usd=size_usd, leverage=LEVERAGE, type='long' if position.type == 0 else 'short')
#                     pnl_at_new_sl = get_pnl_at_price(current_price=position.sl, entry_price=position.price_open, position_size_usd=size_usd, leverage=LEVERAGE, type='long' if position.type == 0 else 'short')
#                     capital_loss_at_old_sl = (pnl_at_old_sl / CAPITAL_PER_TRADE) * 100
#                     capital_loss_at_new_sl= (pnl_at_new_sl / CAPITAL_PER_TRADE) * 100

#                     print(f"{position.symbol} - TRAILING STOP - UNREALIZED PNL: ${position.profit:.3f} ({capital_gain_at_current_unrealized_pnl:.3f}% CAP) - OLD SL: ${old_sl_price:.3f} ({old_sl_price_diff:.3f}%)({capital_loss_at_old_sl:.3f}% CAP) - NEW SL: ${position.sl:.3f} ({new_sl_price_diff:.3f}%)({capital_loss_at_new_sl:.3f}% CAP)")
#                     break  # Exit the loop after adjusting the stop loss
        

#     sleep(60)